In [ ]:
%matplotlib inline
import matplotlib.pyplot as ptl
from PIL import Image
from pathlib import Path
import random
import shutil

def resize_images(source_dir, target_dir, test_samples):
  
  source_dir = Path(source_dir)
  target_dir = Path(target_dir)

  try:
    target_dir.mkdir()
    train_path = Path(target_dir/"train")
    train_path.mkdir()
    
    test_path = Path(target_dir/"test")
    test_path.mkdir()
  except FileExistsError:
    print("Target Directory already exists.")


  for dir in source_dir.iterdir():
    print(f"Directory name: {dir.name}")
    print("Images in directory")
    x = 0
    for image in dir.iterdir():
      print(f"Resizing: {image.name}")
      img = Image.open(image)
      img = img.convert("RGB")
      
#      plt.imshow(img)
#      plt.axis('on')
#      plt.title(image)
#      plt.show()

      img = img.resize((32,32))
#      plt.imshow(img)
#      plt.axis('on')
#      plt.title(image)
#     plt.show()
      img.save(train_path/f"{dir.name}_{x}.jpg")
      x = x+1
    samples = random.sample(range(0, x), test_samples)
    
    for sample in samples:
      print(f"New Test Sample: {dir.name}_{sample}.jpg")
      shutil.move(target_dir/f"train/{dir.name}_{sample}.jpg", target_dir/f"test/{dir.name}_{sample}.jpg")

In [ ]:
resize_images("../data/original", "../data/resized", 10)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

def generate_images(source_path, target_path, generations):
  
  datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    channel_shift_range = 10,
    fill_mode= 'nearest'
  )
  source_path = Path(source_path)
  target_path = Path(target_path)
    
  if target_path.exists():
    print("Target already exists, please delete it.")
    return 0
  
  target_path.mkdir(parents=True)
  for image in source_path.iterdir():
        
    image_name, image_ext = image.name.split(".")
    img = load_img(image)
    img = img_to_array(img)
    img = img.reshape((1,) + img.shape)
    
    i = 0
    for batch in datagen.flow(img, batch_size=1, save_to_dir=target_path, save_prefix=image_name, save_format="jpg"):
      i = i+1
      if i >= generations:
        break

In [ ]:
generate_images("../data/resized/train", "../data/generated/train", 100)
generate_images("../data/resized/test", "../data/generated/test", 100)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from pathlib import Path
categories = ['bulbasaur', 'charmander', 'squirtle']

def img_set(images_path):    
    images = []
    labels = []
    images_path = Path(images_path)
    for img in images_path.iterdir():
        image = load_img(img)
        image = img_to_array(image)
        label = categories.index(img.name.split('_')[0])
        labels.append(label)
        images.append(image)
    return np.array(images), np.array(labels)

In [ ]:
train_dir = "../data/generated/train"
test_dir = "../data/generated/test"

x_train, y_train = img_set(train_dir)
x_test, y_test = img_set(test_dir)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout

X_train = np.array(x_train).astype('float32')
X_test = np.array(x_test).astype('float32')
X_train /= 255
X_test /= 255

num_classes=3
Y_train = to_categorical(y_train, num_classes=num_classes)
Y_test = to_categorical(y_test, num_classes=num_classes)

In [ ]:

model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, input_shape=[num_classes]))
model.add(Activation('softmax'))

optimizer = optimizers.Adam()
model.compile(loss='categorical_crossentropy',
             optimizer=optimizer,
             metrics=['accuracy'])

epochs = 10
batch_size = 150


In [ ]:

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

In [ ]:
from PIL import Image
from pathlib import Path
def recognize_image(image_path):
    categories = ['bulbasaur', 'charmander', 'squirtle']
    image_path = Path(image_path)
    resized_image_path = image_path.parent/f"resized_{image_path.name}"
    img = Image.open(image_path)
    img = img.convert("RGB")
    img = img.resize((32,32))
    img.save(resized_image_path)
    
    image = load_img(resized_image_path)
    image = img_to_array(image)
    img_input_array = np.array([image])
    predictions = model.predict(img_input_array)
    prediction = np.argmax(predictions, axis=1)
    return categories[prediction[0]]

In [ ]:
recognize_image("../charmander.jpg")